# ROBOFLOW SECOND VERSION 

In [1]:
import os
import json
import shutil
import requests

# Constants for API and directories
API_URL = "https://detect.roboflow.com"
API_KEY = "QZ9pAmqgTUAXMUXi7tO3"
MODEL_ID = "weapon-detection2-yyiij-acspc/1"

In [7]:
# Folder paths
image_folder = "/Users/apple/Downloads/01_January"
output_json = "weapon_classification_results.json"
classified_images_folder = "/Users/apple/Downloads/Deteced-images2"

In [8]:
def setup_directories():
    """
    Ensure the output directory for classified images exists.
    """
    os.makedirs(classified_images_folder, exist_ok=True)


In [4]:
def get_images(folder):
    """
    List all .jpg images in the specified folder.

    Args:
        folder (str): Path to the folder containing images.

    Returns:
        list: List of image file paths.
    """
    return [os.path.join(folder, img) for img in os.listdir(folder) if img.endswith(".jpg")]


In [9]:
def infer_image(image_path):
    """
    Perform inference on a single image using the Roboflow API.

    Args:
        image_path (str): Path to the image file.

    Returns:
        dict: The response JSON from the API, or None if an error occurred.
    """
    try:
        with open(image_path, "rb") as image_file:
            response = requests.post(
                f"{API_URL}/{MODEL_ID}",
                params={"api_key": API_KEY},
                files={"file": image_file}
            )
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error for {os.path.basename(image_path)}: {response.status_code} - {response.text}")
                return None
    except Exception as e:
        print(f"Error processing {os.path.basename(image_path)}: {e}")
        return None

In [13]:
def save_intermediate_results(results, summary):
    """
    Save intermediate results and summary to the JSON file.

    Args:
        results (list): List of classification results.
        summary (dict): Summary of classifications.
    """
    with open(output_json, "w") as json_file:
        json.dump({"results": results, "summary": summary}, json_file, indent=4)
    print(f"Intermediate results saved to {output_json}")


In [14]:
def process_images(images, save_interval=1000):
    """
    Process images in chunks, with a pause after processing a defined number of images.

    Args:
        images (list): List of image file paths.
        save_interval (int): Number of images to process before saving and pausing.

    Returns:
        tuple: List of results and classification summary.
    """
    results = []
    summary = {}
    total_images = len(images)

    for index, image_path in enumerate(images, start=1):
        result = infer_image(image_path)
        if result:
            predictions = result.get("predictions", [])
            weapon_classes = [
                pred["class"] for pred in predictions if pred["confidence"] >= 0.5
            ]
            if weapon_classes:
                # Append results
                results.append({
                    "image_name": os.path.basename(image_path),
                    "classes": weapon_classes
                })

                # Copy image to classified folder
                shutil.copy(image_path, classified_images_folder)

                # Update summary
                for weapon_class in weapon_classes:
                    summary[weapon_class] = summary.get(weapon_class, 0) + 1

        # Show progress every 100 images
        if index % 100 == 0 or index == total_images:
            remaining = total_images - index
            print(f"Processed {index}/{total_images} images. {remaining} images remaining.")

        # Save intermediate results every save_interval images
        if index % save_interval == 0 or index == total_images:
            save_intermediate_results(results, summary)

            # Pause for user input
            print(f"Processed {index}/{total_images} images. Pausing for user input.")
            user_input = input("Do you want to continue processing? (yes/no): ").strip().lower()
            if user_input != "yes":
                print("Exiting program...")
                break

    return results, summary

In [15]:
def main():
    """
    Main execution function to organize the workflow.
    """
    print("Setting up directories...")
    setup_directories()

    print("Getting image list...")
    images = get_images(image_folder)
    total_images = len(images)
    print(f"Found {total_images} images to process.")

    print("Processing images...")
    results, summary = process_images(images, save_interval=1000)

    print("Saving final results to JSON...")
    save_intermediate_results(results, summary)

    print(f"Total images identified with weapons: {len(results)}")
    print("Classification summary:")
    for weapon_class, count in summary.items():
        print(f"  {weapon_class}: {count}")


if __name__ == "__main__":
    main()

Setting up directories...
Getting image list...
Found 52967 images to process.
Processing images...
Processed 100/52967 images. 52867 images remaining.
Processed 200/52967 images. 52767 images remaining.
Processed 300/52967 images. 52667 images remaining.
Processed 400/52967 images. 52567 images remaining.
Processed 500/52967 images. 52467 images remaining.
Processed 600/52967 images. 52367 images remaining.
Processed 700/52967 images. 52267 images remaining.
Processed 800/52967 images. 52167 images remaining.
Processed 900/52967 images. 52067 images remaining.
Processed 1000/52967 images. 51967 images remaining.
Intermediate results saved to weapon_classification_results.json
Processed 1000/52967 images. Pausing for user input.
Processed 1100/52967 images. 51867 images remaining.
Processed 1200/52967 images. 51767 images remaining.
Processed 1300/52967 images. 51667 images remaining.
Processed 1400/52967 images. 51567 images remaining.
Processed 1500/52967 images. 51467 images remaini

KeyboardInterrupt: 